### L1 and L2 regularization

recall the equation for a line $y=mx+b$, with multiple features (X), $\hat{y} = \hat{\beta}_0 + \hat{\beta}_1x_1 + \hat{\beta}_2x_2 +\cdots + \hat{\beta}_kx_k$. The estimate for a best fit line is $\hat{y}$

In most cases for linear regression the cost function is minimized. Here the residual $y - \hat{y}$ is both squared and summed over, resulting in the  residual sum of squares (RSS) $ = \sum^{n}_{i=1}(y_i - \hat{y}_i)^2$ If the estimate for a best fit line is substituted into the RSS it becomes
$\sum^{n}_{i=1}(y_i - \hat{y}_i)^2 = \sum^{n}_{i=1}(y_i - \hat{\beta}_0 - \hat{\beta}_1x_{i1} - \hat{\beta}_2x_{i1} - \cdots \hat{\beta}_kx_{ik})$

So then RSS is $\sum^{n}_{i=1}(y_i - \beta_o - \sum^{k}_{j=1}\beta_j x_{ij})^2$ so that a penalty term can be added to the expression 
$\sum^{n}_{i=1}(y_i - \beta_o - \sum^{k}_{j=1}\beta_j x_{ij})^2 + \lambda\sum^{k}_{j=1}\beta^{2}_{j} $ The added penalty is a squared term that is tunable by a hyperparameter $\lambda$ This is the L2 penalty, ridge regression

Likewise, a similar penalty can also be added that factors the absolute value of the coefficients $\sum^{n}_{i=1}(y_i - \beta_o - \sum^{k}_{j=1}\beta_j x_{ij})^2 + \lambda\sum^{k}_{j=1}|\beta_{j}|$ Called the L1 regularization, or least absolute shrinkage and selection operator LASSO.

Finally, we can add both penalty terms, called an elastic net RSS + L2 + L1.

This notebook explores the different penalty terms using the ISLR advertising data, to see how the penalty terms shrink insignificant coefficients to zero in higher order fits.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('Advertising.csv')

In [4]:
X = df.drop('sales',axis=1)

In [5]:
y = df['sales']

In [6]:
from sklearn.preprocessing import PolynomialFeatures

In [7]:
polynomial_converter = PolynomialFeatures(degree=3,include_bias=False)

In [8]:
poly_features = polynomial_converter.fit_transform(X)

In [9]:
poly_features.shape

(200, 19)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

In [12]:
#scale data
from sklearn.preprocessing import StandardScaler

In [13]:
scaler = StandardScaler()

In [14]:
scaler.fit(X_train)

StandardScaler()

In [15]:
X_train = scaler.transform(X_train)

In [16]:
X_test = scaler.transform(X_test)

In [17]:
X_train[0]

array([ 0.49300171, -0.33994238,  1.61586707,  0.28407363, -0.02568776,
        1.49677566, -0.59023161,  0.41659155,  1.6137853 ,  0.08057172,
       -0.05392229,  1.01524393, -0.36986163,  0.52457967,  1.48737034,
       -0.66096022, -0.16360242,  0.54694754,  1.37075536])

In [18]:
#ridge regression
from sklearn.linear_model import Ridge

In [1]:
#help(Ridge)

In [20]:
ridge_model = Ridge(alpha=10)

In [21]:
ridge_model.fit(X_train,y_train)

Ridge(alpha=10)

In [22]:
test_predictions = ridge_model.predict(X_test)

In [23]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [24]:
MAE = mean_absolute_error(y_test,test_predictions)

In [25]:
MAE

0.5774404204714169

In [26]:
RMSE = np.sqrt(mean_squared_error(y_test,test_predictions))

In [27]:
RMSE

0.8946386461319674

In [28]:
#with cross validation
from sklearn.linear_model import RidgeCV

In [29]:
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10.0),scoring='neg_mean_absolute_error')

In [30]:
ridge_cv_model.fit(X_train,y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), scoring='neg_mean_absolute_error')

In [31]:
ridge_cv_model.alpha_

0.1

In [32]:
#from sklearn.metrics import SCORERS

In [33]:
#use this if you forget or dont know a specific scoring method call
#SCORERS.keys()

In [34]:
test_predictions = ridge_cv_model.predict(X_test)

In [35]:
MAE = mean_absolute_error(y_test,test_predictions)

In [36]:
RMSE = np.sqrt(mean_squared_error(y_test,test_predictions))

In [37]:
MAE

0.427377488435447

In [38]:
RMSE

0.6180719926966545

In [39]:
ridge_cv_model.coef_

array([ 5.40769392,  0.5885865 ,  0.40390395, -6.18263924,  4.59607939,
       -1.18789654, -1.15200458,  0.57837796, -0.1261586 ,  2.5569777 ,
       -1.38900471,  0.86059434,  0.72219553, -0.26129256,  0.17870787,
        0.44353612, -0.21362436, -0.04622473, -0.06441449])

In [40]:
ridge_cv_model.best_score_

-0.37492233402929626

In [41]:
#LASSO - least Absolute Shrinkage and Selection Operator

In [43]:
from sklearn.linear_model import LassoCV

In [44]:
#cross validation give a range of possible alphas (hyperparameters)

In [52]:
lasso_cv_model = LassoCV(eps=0.1,n_alphas=100,cv=5)

In [53]:
lasso_cv_model.fit(X_train,y_train)

LassoCV(cv=5, eps=0.1)

In [54]:
lasso_cv_model.alpha_

0.49430709092258285

In [55]:
test_predictions = lasso_cv_model.predict(X_test)

In [56]:
MAE = mean_absolute_error(y_test,test_predictions)

In [57]:
RMSE = np.sqrt(mean_squared_error(y_test,test_predictions))

In [58]:
MAE

0.6541723161252858

In [59]:
RMSE

1.130800102276254

In [61]:
lasso_cv_model.coef_
#wow, this is only considering 2 features (all zeros are ignored features)
#at cost of higher RMSE, we are able to constrain number of features
#as we search for more alpha values, this will become more complex and 
#consider more features (eps=0.001,max_iteration=10000000)

array([1.002651  , 0.        , 0.        , 0.        , 3.79745279,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

In [62]:
#elastic net

In [70]:
from sklearn.linear_model import ElasticNetCV

In [71]:
elastic_model = ElasticNetCV(l1_ratio=[.1,.5,.7,.9,.95,.99,.1],
                            eps=0.001,n_alphas=100,max_iter=1000000)

In [72]:
elastic_model.fit(X_train,y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 0.1], max_iter=1000000)

In [73]:
elastic_model.l1_ratio_

0.99

In [74]:
elastic_model.alpha_

0.004993000918409929

In [75]:
lasso_cv_model.alpha_

0.49430709092258285

In [76]:
test_predictions = elastic_model.predict(X_test)

In [77]:
MAE = mean_absolute_error(y_test,test_predictions)

In [78]:
RMSE = np.sqrt(mean_squared_error(y_test,test_predictions))

In [79]:
MAE

0.4361938933662325

In [80]:
RMSE

0.6104106917384732